In [2]:
import pandas as pd
import numpy as np

import json
import requests
from bs4 import BeautifulSoup as bts
import bs4
import requests
import time
import re

In [829]:
pd.options.display.max_colwidth = 9999

In [830]:
def getAndParseURL(url):
    result = requests.get(url,headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [912]:
def harvest_rentals(rentals, result):
    for rentallink in rentals:
        html = getAndParseURL(rentallink)
        
        try:
            name = html.find('h1', class_= 'fontRB').text.strip()
        except:
            name = np.nan
                
        try:
            price = html.find('p', class_ = 'fz24-text price').text.strip().strip(" TL\n                  ")
        except:
            price = np.nan
        
        area_m2 = html.find("div", class_= "container det-container").find(string=re.compile("Brüt / Net M2")).findNext().text.replace("m2","").strip()
        #ad_no = html.find("div", class_= "container det-container").find(string=re.compile("İlan no")).findNext().text.strip()
        date = html.find("div", class_= "container det-container").find(string=re.compile("Son Güncelleme Tarihi")).findNext().text.strip()
        floor_number = html.find("div", class_= "container det-container").find(string=re.compile("Bulunduğu Kat")).findNext().text.replace(". Kat","").strip()
        age = html.find("div", class_= "container det-container").find(string=re.compile("Bina Yaşı")).findNext().text.replace(" Yaşında","").strip()
        bath_number = html.find("div", class_= "container det-container").find(string=re.compile("Banyo Sayısı")).findNext().text.strip()
        
        di = []
        district = html.find("div", {"class":"det-title-bottom"})
        for i in district:
            di.append(i.findAll()[2].text.strip())
            
            
        room_number = []
        room = html.find("div", {"class":"det-title-bottom"})
        for i in room:
            room_number.append(i.findAll()[5].text.strip())
            
        
        tot_fl_nr = []
        total_floor_number = html.find("div", {"class":"container det-container"}).find(string=re.compile("Kat Sayısı")).findNext().text.strip(" Katlı")
        for i in total_floor_number:
            tot_fl_nr.append(i)
            
            
        result.append([name, price, area_m2, date, floor_number, age, bath_number, di, room_number, tot_fl_nr])
        time.sleep(5)

In [913]:
result = []
rentals = []
for i in range(0, 8):
    start = i+1
    html = getAndParseURL("https://www.hepsiemlak.com/manisa-kiralik/daire?page=% s" % start)
    #html = getAndParseURL("https://www.hepsiemlak.com/yunusemre-kiralik/daire?buildingAges=6-10,11-15")
    for sonuc in html.findAll('div', class_= 'links'):              
        rentals.append("https://www.hepsiemlak.com"+ sonuc.a['href']) #linkleri topluyor ve yukarıdaki def.te her bir linkin içine giriyor  
            
    harvest_rentals(rentals, result)   
    
column = ["name", "price", "area_m2", "date", "floor_number", "age", "bath_number", "district", "room_number", "total_floor_number"]
dfMatch_unique = pd.DataFrame.from_records(result, columns=column)

In [915]:
dfMatch_unique.sample(5)

,name,price,area_m2,date,floor_number,age,bath_number,district,room_number,total_floor_number
604,AKHİSAR GÜRBÜZ EMLAK TAN KETHÜDA MAHALLESİ HERGELEN MEYDANINDA 5.KAT SOBALI KİRALIK DAİRE,4.000,127,13-11-2023,En Üst Kat,30,1,[Kethüda],[3 + 1],[5]
605,Eski garajın orada kiralık ev Dinçer Mahallesi,8.000,140,09-11-2023,2,18,1,[Dinçer],[3 + 1],[4]
58,M / Ş gayrumenkulden kiralık daire,7.500,120,09-11-2023,Kot 1,15,1,[Eski Cami],[3 + 1],[4]
250,KİRALIK 2+1 100 m2MURADİYE MAH. Önü Açık Ferah Bir Daire,12.000,105,08-11-2023,2,6,1,[Muradiye],[2 + 1],[3]
585,Kiralik istiklal mh limoncu cami civari zemin kat 2+1 85m2,5.000,90,11-11-2023,Zemin,25,1,[İstiklal],[2 + 1],[4]


In [916]:
dfMatch_unique.to_csv("dfMatch_unique.csv")

In [425]:
df = pd.read_csv("dfMatch_unique.csv", index_col=0)

# Preprocessing

In [426]:
df.head()

,name,price,area_m2,date,floor_number,age,bath_number,district,room_number,total_floor_number
0,KİRALIK 3+1 MANİSA AKGEDİK 2. KAT ARAKAT 3+1 1...,13.000,130.0,13-11-2023,2,4,1,['Akgedik'],['3 + 1'],['4']
1,KİRALIK 3+1 140 m22. Kat Full özellikli İBRAHİ...,14.000,145.0,13-11-2023,2,17,1,['İbrahimçelebi'],['3 + 1'],['5']
2,MANİSA MERKEZDE EŞYALI 1+1 KİRALIK DAİRE,9.750,75.0,13-11-2023,Yüksek Giriş,30,1,['Mimar Sinan'],['1 + 1'],['5']
3,Kiralık Eşyalı Daire Manisa Karaali villaların...,15.500,85.0,13-11-2023,Ara Kat,3,1,['Karaali'],['1 + 1'],['3']
4,Kiralık Daire Manisa 2.Anafartalar Mah,12.500,90.0,13-11-2023,3,28,1,['2. Anafartalar'],['2 + 1'],['6']


In [427]:
df.shape

(864, 10)

In [428]:
df.columns

Index(['name', 'price', 'area_m2', 'date', 'floor_number', 'age',
       'bath_number', 'district', 'room_number', 'total_floor_number'],
      dtype='object')

In [429]:
df.drop_duplicates(inplace=True, ignore_index=True)

In [430]:
df.shape

(192, 10)

In [431]:
df.total_floor_number.isnull().any()

False

In [432]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                192 non-null    object 
 1   price               192 non-null    object 
 2   area_m2             192 non-null    float64
 3   date                192 non-null    object 
 4   floor_number        192 non-null    object 
 5   age                 192 non-null    object 
 6   bath_number         192 non-null    int64  
 7   district            192 non-null    object 
 8   room_number         192 non-null    object 
 9   total_floor_number  192 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 15.1+ KB


## floor_number and total_floor_number

In [433]:
for i in df.total_floor_number:
    i = str(i)
    
#working with strings are easier than lists

In [434]:
liste1 = []
for i in df.total_floor_number:
    if len(i) == 10:
        i = np.nan
        liste1.append(i)
    elif len(i) != 10:
        liste1.append(i)
        
df["total_floor_number"] = liste1

In [435]:
df["total_floor_number"] = df["total_floor_number"].str.strip("[").str.strip("]")
df["total_floor_number"] = df["total_floor_number"].str.strip("'").str.strip("'")
df["total_floor_number"] = df["total_floor_number"].astype(float)

In [436]:
df["total_floor_number"].isnull().any()

True

In [437]:
df["total_floor_number"].value_counts()

5.0    56
4.0    36
3.0    27
2.0    25
6.0    12
8.0     6
7.0     3
1.0     1
Name: total_floor_number, dtype: int64

In [438]:
df[df.total_floor_number.isnull()]

#As apperance, generally floor_number is high when total_floor_number is NaN

,name,price,area_m2,date,floor_number,age,bath_number,district,room_number,total_floor_number
14,Akhisar Atatürk Mahallesi'nde 3+1 Kiralık Daire,13.000,138.0,13-11-2023,4,1,2,['Atatürk'],['3 + 1'],NaN
30,GÜZELYURT MAH.EŞYALI KİRALIK 1+1,12.000,75.0,02-11-2023,10,5,1,['Güzelyurt'],['1 + 1'],NaN
37,Manisa Meydan projesinde Kiralık 1+1 Bahçe Katı,12.500,45.0,23-09-2023,Bahçe Katı,Sıfır Bina,1,['Uncubozköy'],['1 + 1'],NaN
45,Manisa Meydan Sitesi Yarı Eşyalı 1+1 Daire Kir...,11.000,48.0,11-11-2023,9,Sıfır Bina,1,['Uncubozköy'],['1 + 1'],NaN
46,Manisa Meydan sitesinde 1+1 kiralık daire,14.000,56.0,06-11-2023,9,Sıfır Bina,1,['Uncubozköy'],['1 + 1'],NaN
52,Manisa Startkey Elit2'den Park Manzara Evleri ...,20.000,160.0,07-11-2023,9,7,2,['Cumhuriyet'],['3 + 1'],NaN
60,ÖZGÜN GM'DEN MANİSA MEYDAN'DA EŞYALI KİRALIK 1+1,14.000,45.0,09-11-2023,9,Sıfır Bina,1,['Uncubozköy'],['1 + 1'],NaN
84,MANİSA MEYDAN DOĞA VE ŞEHİR MANZARALI EŞYALI 1...,14.000,45.0,06-11-2023,11,Sıfır Bina,1,['Uncubozköy'],['1 + 1'],NaN
89,BEYZA GM'DEN MANİSA MEYDAN PROJESİNDE EŞYALI K...,13.500,45.0,15-10-2023,8,Sıfır Bina,1,['Uncubozköy'],['1 + 1'],NaN
96,MANİSA MEYDAN HAVUZ VE PEYZAJ MANZARALI FIRSAT...,12.000,45.0,02-11-2023,Ara Kat,Sıfır Bina,1,['Uncubozköy'],['1 + 1'],NaN


In [439]:
cond = ((df.floor_number == "Bahçe Katı") | (df.floor_number == "Yüksek Giriş") | (df.floor_number == "Zemin") | (df.floor_number == "Kot 1") | (df.floor_number == "Villa Katı") | (df.floor_number == "Yarı Bodrum"))

df.loc[cond, "floor_number"] = 1

In [440]:
cond = (df.floor_number == "Kot 3")

df.loc[cond, "floor_number"] = 3

In [441]:
df["floor_number"].value_counts()

# For middle floor (Ara kat), I need int(total_floor_number/2)
# For top floor (En üst kat) and attic (Çatı katı), I need only total_floor_number

2             41
Ara Kat       30
1             26
3             23
1             18
4             12
En Üst Kat    12
5              8
6              4
9              4
8              2
13             2
7              2
11             2
10             2
12             1
16             1
Çatı Katı      1
3              1
Name: floor_number, dtype: int64

In [497]:
df[df["floor_number"] == 16]

,name,price,area_m2,date,floor_number,age,bath_number,district,room_number,total_floor_number,age_new,district_new,room_number_new,area_new
106,3+1 16.kat full 170 M2 kiralık,9.0,170.0,12-11-2023,16.0,0.0,2,Hürriyet,3 + 1,4.0,0.0,2.0,3.0,6.0


In [442]:
cond = df["floor_number"] == "Ara Kat"

df.loc[cond, "floor_number"] = (df["total_floor_number"]/2).round()

In [443]:
cond = ((df["floor_number"] == "En Üst Kat") | (df["floor_number"] == "Çatı Katı"))

df.loc[cond, "floor_number"] = df["total_floor_number"]

In [444]:
df["floor_number"] = df["floor_number"].fillna(df["floor_number"].median())

df["floor_number"] = df["floor_number"].astype("float")

In [445]:
df["floor_number"].value_counts()

2.0     66
1.0     44
3.0     27
5.0     18
4.0     16
6.0      5
9.0      4
7.0      2
13.0     2
8.0      2
11.0     2
10.0     2
12.0     1
16.0     1
Name: floor_number, dtype: int64

In [446]:
df.total_floor_number = df.total_floor_number.fillna(df.total_floor_number.median())

## age

In [447]:
df["age"].value_counts()

Sıfır Bina    51
25            19
20            16
5             13
4             10
15            10
30            10
3             10
18             6
1              6
10             5
7              5
6              3
35             3
8              3
11             3
17             2
13             2
12             2
26             2
2              2
22             1
16             1
14             1
40             1
9              1
27             1
21             1
23             1
28             1
Name: age, dtype: int64

In [448]:
cond = (df["age"] == "Sıfır Bina")

df.loc[cond, "age"] = "0"

In [449]:
df["age"] = df["age"].astype(float)

In [450]:
cond = (df["age"] <= 5)

df.loc[cond, "age_new"] = 0

In [451]:
cond = ((df["age"] > 5) & (df["age"] <= 10))

df.loc[cond, "age_new"] = 1

In [452]:
cond = ((df["age"] > 10) & (df["age"] <= 15))

df.loc[cond, "age_new"] = 2

In [453]:
cond = ((df["age"] > 15) & (df["age"] <= 20))

df.loc[cond, "age_new"] = 3

In [454]:
cond = ((df["age"] > 20) & (df["age"] <= 25))

df.loc[cond, "age_new"] = 4

In [455]:
cond = (df["age"] > 25)

df.loc[cond, "age_new"] = 5

In [456]:
df.age_new.value_counts()

0.0    92
3.0    25
4.0    22
2.0    18
5.0    18
1.0    17
Name: age_new, dtype: int64

## price

In [457]:
df.head()

,name,price,area_m2,date,floor_number,age,bath_number,district,room_number,total_floor_number,age_new
0,KİRALIK 3+1 MANİSA AKGEDİK 2. KAT ARAKAT 3+1 1...,13.000,130.0,13-11-2023,2.0,4.0,1,['Akgedik'],['3 + 1'],4.0,0.0
1,KİRALIK 3+1 140 m22. Kat Full özellikli İBRAHİ...,14.000,145.0,13-11-2023,2.0,17.0,1,['İbrahimçelebi'],['3 + 1'],5.0,3.0
2,MANİSA MERKEZDE EŞYALI 1+1 KİRALIK DAİRE,9.750,75.0,13-11-2023,1.0,30.0,1,['Mimar Sinan'],['1 + 1'],5.0,5.0
3,Kiralık Eşyalı Daire Manisa Karaali villaların...,15.500,85.0,13-11-2023,2.0,3.0,1,['Karaali'],['1 + 1'],3.0,0.0
4,Kiralık Daire Manisa 2.Anafartalar Mah,12.500,90.0,13-11-2023,3.0,28.0,1,['2. Anafartalar'],['2 + 1'],6.0,5.0


In [459]:
df.price.value_counts()

12.000       32
10.000       17
15.000       14
12.500       13
11.000       12
13.000       11
14.000       10
8.000         7
10.500        6
7.500         5
20.000        5
9.500         5
9.000         5
7.000         5
17.000        5
25.000        4
13.500        4
11.500        3
6.000         3
18.000        3
8.500         2
16.000        2
6.500         2
15.500        2
5.500         1
12.750        1
15.750        1
14.500        1
12.250        1
22.000        1
9.750         1
4.000         1
18.500        1
5.000         1
23.000        1
35.000        1
14.750        1
1.650.000     1
8.250         1
Name: price, dtype: int64

In [460]:
df[df.price == "1.650.000"]

,name,price,area_m2,date,floor_number,age,bath_number,district,room_number,total_floor_number,age_new
184,satılık daire resatbeyde,1.650.000,100.0,16-09-2023,4.0,3.0,1,['Reşat Bey'],['2 + 1'],4.0,0.0


In [461]:
df.drop([184], inplace=True)

In [462]:
df.price = df.price.astype(float)

In [463]:
df.head()

,name,price,area_m2,date,floor_number,age,bath_number,district,room_number,total_floor_number,age_new
0,KİRALIK 3+1 MANİSA AKGEDİK 2. KAT ARAKAT 3+1 1...,13.00,130.0,13-11-2023,2.0,4.0,1,['Akgedik'],['3 + 1'],4.0,0.0
1,KİRALIK 3+1 140 m22. Kat Full özellikli İBRAHİ...,14.00,145.0,13-11-2023,2.0,17.0,1,['İbrahimçelebi'],['3 + 1'],5.0,3.0
2,MANİSA MERKEZDE EŞYALI 1+1 KİRALIK DAİRE,9.75,75.0,13-11-2023,1.0,30.0,1,['Mimar Sinan'],['1 + 1'],5.0,5.0
3,Kiralık Eşyalı Daire Manisa Karaali villaların...,15.50,85.0,13-11-2023,2.0,3.0,1,['Karaali'],['1 + 1'],3.0,0.0
4,Kiralık Daire Manisa 2.Anafartalar Mah,12.50,90.0,13-11-2023,3.0,28.0,1,['2. Anafartalar'],['2 + 1'],6.0,5.0


## district

In [464]:
for i in df.district:
    i = str(i)

In [465]:
df["district"] = df["district"].str.strip("[").str.strip("]")
df["district"] = df["district"].str.strip("'").str.strip("'")

In [466]:
df.district.value_counts()

Uncubozköy        21
75. Yıl           20
Muradiye          15
Atatürk            8
Güzelyurt          7
Merkez Efendi      7
Kurtuluş           6
Hürriyet           6
Şehitler           5
Karaali            4
Selvilitepe        4
Paşa               4
Akgedik            4
Yörük              4
Cumhuriyet         4
1. Anafartalar     3
Ragıp Bey          3
2. Anafartalar     3
İstasyon           3
Yeni Mahalle       3
Mimar Sinan        3
Tevfikiye          3
Sakarya            3
Peker              3
Keçili Köy         3
Eski Cami          2
Kethüda            2
Göktaşlı           2
Yarhasanlar        2
Ege                2
Hafsa Sultan       2
Fatih              2
Akmescit           2
Mesir              2
Yılmazlar          2
Laleli             2
Ergenekon          2
Akıncılar          2
İbrahimçelebi      2
Namık Kemal        1
Yeni               1
Alaybey            1
Sağlık             1
Kırveli            1
Bağyolu            1
İstiklal           1
50. Yıl            1
Dere         

In [467]:
df.groupby(["district"])[["price"]].mean().sort_values("price", ascending = False)

,price
district,
Bağyolu,18.000000
Güzelyurt,17.892857
Peker,17.000000
Cumhuriyet,16.750000
Yeni Mahalle,15.666667
1. Anafartalar,15.500000
Laleli,15.250000
Dere,15.000000
75. Yıl,14.950000


In [468]:
cond = ((df["district"] == "Bağyolu") | (df["district"] == "Güzelyurt") | (df["district"] == "Peker") | (df["district"] == "Cumhuriyet") | (df["district"] == "Yeni Mahalle") | (df["district"] == "1. Anafartalar") | (df["district"] == "Laleli") | (df["district"] == "Dere"))

df.loc[cond, "district_new"] = 0

In [469]:
cond = ((df["district"] == "75. Yıl") | (df["district"] == "Tevfikiye") | (df["district"] == "Uncubozköy") | (df["district"] == "Muradiye") | (df["district"] == "Akmescit") | (df["district"] == "Sakarya") | (df["district"] == "Akıncılar") | (df["district"] == "Karaali") | (df["district"] == "Yörük") | (df["district"] == "2. Anafartalar") | (df["district"] == "Keçili Köy") | (df["district"] == "Ergenekon") | (df["district"] == "Mimar Sinan") | (df["district"] == "İbrahimçelebi") | (df["district"] == "Akgedik") | (df["district"] == "Kırveli") | (df["district"] == "Adakale")  )

df.loc[cond, "district_new"] = 1

In [470]:
cond = ((df["district"] == "Kurtuluş") | (df["district"] == "Atatürk") | (df["district"] == "Yarhasanlar") | (df["district"] == "Merkez Efendi") | (df["district"] == "Eski Cami") | (df["district"] == "Ege") | (df["district"] == "Alaybey") | (df["district"] == "Göktaşlı") | (df["district"] == "Yeni") | (df["district"] == "Paşa") | (df["district"] == "Hafsa Sultan") | (df["district"] == "Mesir") | (df["district"] == "Şehitler") | (df["district"] == "Yılmazlar") | (df["district"] == "Selvilitepe") | (df["district"] == "Hürriyet") | (df["district"] == "50. Yıl")  )

df.loc[cond, "district_new"] = 2

In [471]:
cond = ((df["district"] == "Fatih") | (df["district"] == "İstasyon") | (df["district"] == "Sağlık") | (df["district"] == "Dinçer") | (df["district"] == "Namık Kemal") | (df["district"] == "Reşat Bey") | (df["district"] == "Ragıp Bey") | (df["district"] == "Seyitali") | (df["district"] == "Kethüda") | (df["district"] == "Sancaklıbozköy") | (df["district"] == "İstiklal")) 

df.loc[cond, "district_new"] = 3

## room number

In [472]:
for i in df.room_number:
    i = str(i)

In [473]:
df["room_number"] = df["room_number"].str.strip("[").str.strip("]")
df["room_number"] = df["room_number"].str.strip("'").str.strip("'")

In [474]:
df["room_number"].value_counts()

3 + 1    90
2 + 1    50
1 + 1    46
4 + 1     2
1 + 0     1
5 + 2     1
4 + 2     1
Name: room_number, dtype: int64

In [490]:
cond = (df["room_number"] == "1 + 0")

df.loc[cond, "room_number_new"] = 0

In [491]:
cond = (df["room_number"] == "1 + 1")

df.loc[cond, "room_number_new"] = 1

In [492]:
cond = (df["room_number"] == "2 + 1")

df.loc[cond, "room_number_new"] = 2

In [493]:
cond = (df["room_number"] == "3 + 1")

df.loc[cond, "room_number_new"] = 3

In [494]:
cond = ((df["room_number"] == "4 + 1") | (df["room_number"] == "5 + 2") | (df["room_number"] == "4 + 2"))

df.loc[cond, "room_number_new"] = 4

In [495]:
df["room_number_new"].value_counts()

3.0    90
2.0    50
1.0    46
4.0     4
0.0     1
Name: room_number_new, dtype: int64

In [480]:
df.head()

,name,price,area_m2,date,floor_number,age,bath_number,district,room_number,total_floor_number,age_new,district_new,room_number_new
0,KİRALIK 3+1 MANİSA AKGEDİK 2. KAT ARAKAT 3+1 1...,13.00,130.0,13-11-2023,2.0,4.0,1,Akgedik,3 + 1,4.0,0.0,1.0,3.0
1,KİRALIK 3+1 140 m22. Kat Full özellikli İBRAHİ...,14.00,145.0,13-11-2023,2.0,17.0,1,İbrahimçelebi,3 + 1,5.0,3.0,1.0,3.0
2,MANİSA MERKEZDE EŞYALI 1+1 KİRALIK DAİRE,9.75,75.0,13-11-2023,1.0,30.0,1,Mimar Sinan,1 + 1,5.0,5.0,1.0,1.0
3,Kiralık Eşyalı Daire Manisa Karaali villaların...,15.50,85.0,13-11-2023,2.0,3.0,1,Karaali,1 + 1,3.0,0.0,1.0,1.0
4,Kiralık Daire Manisa 2.Anafartalar Mah,12.50,90.0,13-11-2023,3.0,28.0,1,2. Anafartalar,2 + 1,6.0,5.0,1.0,2.0


In [481]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191 entries, 0 to 191
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                191 non-null    object 
 1   price               191 non-null    float64
 2   area_m2             191 non-null    float64
 3   date                191 non-null    object 
 4   floor_number        191 non-null    float64
 5   age                 191 non-null    float64
 6   bath_number         191 non-null    int64  
 7   district            191 non-null    object 
 8   room_number         191 non-null    object 
 9   total_floor_number  191 non-null    float64
 10  age_new             191 non-null    float64
 11  district_new        191 non-null    float64
 12  room_number_new     191 non-null    float64
dtypes: float64(8), int64(1), object(4)
memory usage: 20.9+ KB


In [483]:
cond1 = (df.area_m2 >= 145)
cond2 = ((df.area_m2 >= 130) & (df.area_m2 < 145))
cond3 = ((df.area_m2 >= 100) & (df.area_m2 < 130))
cond4 = ((df.area_m2 >= 90) & (df.area_m2 < 100))
cond5 = ((df.area_m2 >= 70) & (df.area_m2 < 90))
cond6 = ((df.area_m2 >= 60) & (df.area_m2 < 70))
cond7 = (df.area_m2 < 60)

df.loc[cond1, "area_new"] = 6
df.loc[cond2, "area_new"] = 5
df.loc[cond3, "area_new"] = 4
df.loc[cond4, "area_new"] = 3
df.loc[cond5, "area_new"] = 2
df.loc[cond6, "area_new"] = 1
df.loc[cond7, "area_new"] = 0

In [484]:
df.bath_number.value_counts()

1    165
2     25
3      1
Name: bath_number, dtype: int64

In [485]:
df.head()

,name,price,area_m2,date,floor_number,age,bath_number,district,room_number,total_floor_number,age_new,district_new,room_number_new,area_new
0,KİRALIK 3+1 MANİSA AKGEDİK 2. KAT ARAKAT 3+1 1...,13.00,130.0,13-11-2023,2.0,4.0,1,Akgedik,3 + 1,4.0,0.0,1.0,3.0,5.0
1,KİRALIK 3+1 140 m22. Kat Full özellikli İBRAHİ...,14.00,145.0,13-11-2023,2.0,17.0,1,İbrahimçelebi,3 + 1,5.0,3.0,1.0,3.0,6.0
2,MANİSA MERKEZDE EŞYALI 1+1 KİRALIK DAİRE,9.75,75.0,13-11-2023,1.0,30.0,1,Mimar Sinan,1 + 1,5.0,5.0,1.0,1.0,2.0
3,Kiralık Eşyalı Daire Manisa Karaali villaların...,15.50,85.0,13-11-2023,2.0,3.0,1,Karaali,1 + 1,3.0,0.0,1.0,1.0,2.0
4,Kiralık Daire Manisa 2.Anafartalar Mah,12.50,90.0,13-11-2023,3.0,28.0,1,2. Anafartalar,2 + 1,6.0,5.0,1.0,2.0,3.0


In [486]:
df.isnull().any()

name                  False
price                 False
area_m2               False
date                  False
floor_number          False
age                   False
bath_number           False
district              False
room_number           False
total_floor_number    False
age_new               False
district_new          False
room_number_new       False
area_new              False
dtype: bool

In [496]:
df.to_csv("df_hepsiemlak_cleaned.csv")